In [1]:
from pywinauto.application import Application
from pywinauto.keyboard import SendKeys
import pyautogui
import requests
import urllib3
import time
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
class Region:
    def __init__(self):
        (self.screen_width, self.screen_height) = pyautogui.size()
        self.sys_tray_taskbar = (self.screen_width-500,self.screen_height-30,400,30)
        self.sys_tray_panel = (self.screen_width-500, self.screen_height-250, 500, 250)

In [3]:
class SSR:
    current_server = None
    failed_servers = set()
    inland_servers = set()
    proxy = {
              "http": "http://127.0.0.1:8080",
              "https": "https://127.0.0.1:8080"
            }
    def __init__(self):
        self.region = Region()
        self.app = Application(backend="uia").connect(path="E:\软件\网络\ssr-win\ShadowsocksR-dotnet4.0.exe")
        self.switch_mode(mode='global')
        self.get_current_server()
        print(self.current_server)
        
    def open_ssr_menu(self):
        position = pyautogui.position()
        location = pyautogui.locateCenterOnScreen('./images/sys_tray_toggle.png', confidence=0.9, grayscale=True, region=self.region.sys_tray_taskbar)
        pyautogui.click(location)
        location = pyautogui.locateCenterOnScreen('./images/ssr_icon.png', confidence=0.9, grayscale=True, region=self.region.sys_tray_panel)
        pyautogui.rightClick(location)
        pyautogui.moveTo(position)
        
    def switch_mode(self, mode='pac'): # direct, pac, global
        self.open_ssr_menu()
        self.app.Menu['系统代理模式'].select()
        if mode == 'pac':
            self.app.window(title="系统代理模式")['PAC 模式'].select()
        elif mode == 'global':
            self.app.Menu['全局模式'].select()
        elif mode == 'direct':
            self.app.Menu['直连模式'].select()
        self.current_mode = mode
        
    def update(direct=False):
        self.open_ssr_menu()
        self.app.Menu['服务器订阅'].select()
        if direct:
            self.app.window(title="服务器订阅")['更新SSR服务器订阅（不通过代理）'].select()
        else:
            self.app.window(title="服务器订阅")['更新SSR服务器订阅'].select()
            
    def get_current_server(self):
        if self.current_server:
            return self.current_server
        
        self.open_ssr_menu()
        self.app.Menu['服务器'].select()
        self.app.window(title="服务器").MenuItem2.select()
        items = self.app.window(best_match="OSOECO", control_type="Menu").children()
        for item in items:
            if self.is_server_menu_item_selected(item):
                self.current_server = item.window_text()
        SendKeys('{ESC}')
        SendKeys('{ESC}')
        SendKeys('{ESC}')
        return self.current_server
    
    def is_server_menu_item_selected(self, mi):
        rect = mi.rectangle()
        region = (rect.left, rect.top, rect.width(), rect.height())
        location = pyautogui.locateCenterOnScreen('./images/selected_server.png', confidence=0.9, grayscale=True, region=region)
        if location:
            return True
        else:
            return False
        
    def switch_to_next_server(self):
        self.open_ssr_menu()
        self.app.Menu['服务器'].select()
        self.app.window(title="服务器").MenuItem2.select()
        items = self.app.window(best_match="OSOECO", control_type="Menu").children()
        current_server = self.get_current_server()
        next_item = None
        for idx, item in enumerate(items):
            if current_server is None:
                next_item = item
                break
            elif self.current_server == item.window_text():
                next_idx = idx + 1
                if next_idx == len(items):
                    next_idx = 0
                next_item = items[next_idx]
                break
        self.current_server = next_item.window_text()
        next_item.select()
        
    def is_current_server_on(self):
        response = None
        t1 = time.time()
        try:
            response = requests.get("https://api.ipgeolocation.io/ipgeo?apiKey=8729c2eb97dc4478b00f798d549c78df", proxies=self.proxy, timeout=4, verify=False)
            if self.current_server in self.failed_servers:
                self.failed_servers.remove(self.current_server)
        except Exception as e:
            self.failed_servers.add(self.current_server)
        t2 = time.time()
        if response:
            ipgeo = json.loads(response.text)
            if ipgeo['country_name'].lower() == 'china':
                self.inland_servers.add(ssr.current_server)
            print(t2-t1, ' -- ', ipgeo['ip'], ipgeo['country_name'], ipgeo['city'], ipgeo['time_zone']['name'], '  ---  ', ssr.current_server.split(' (')[0])
            return True
        else:
            print(t2-t1, ' -- ', ssr.current_server)
            return False
        
    def switch_to_next_working_foreign_server(self):
        while True:
            self.switch_to_next_server()
            if '国内' in self.current_server:
                continue
            if self.is_current_server_on():
                break

In [4]:
ssr = SSR()

美国MC - 无限流量 / 三网适用 (us-mc.osoblanco.xyz:4936)


In [8]:
ssr.is_current_server_on()

1.9349126815795898  --  133.18.208.210 Japan Chiyoda Asia/Tokyo   ---   日本IIJ - 三网适用


True

In [28]:
ssr.switch_mode(mode='pac')

In [7]:
ssr.switch_to_next_working_foreign_server()

3.3868279457092285  --  133.18.208.210 Japan Chiyoda Asia/Tokyo   ---   日本IIJ - 三网适用


In [13]:
for i in range(22):
    print(i, end=' => ')
    ssr.is_current_server_on()
    ssr.switch_to_next_server()
    time.sleep(1)
ssr.failed_servers

0 => 1.2021491527557373  --  185.199.225.78 United States Chicago America/Chicago   ---   美国MC - 无限流量 / 三网适用
1 => server failed SOCKSHTTPSConnectionPool(host='api.ipgeolocation.io', port=443): Read timed out. (read timeout=4)
5.18377161026001  --  日本IIJ - 三网适用 (jp-iij.osoblanco.xyz:4936)
2 => server failed SOCKSHTTPSConnectionPool(host='api.ipgeolocation.io', port=443): Max retries exceeded with url: /ipgeo?apiKey=8729c2eb97dc4478b00f798d549c78df (Caused by ConnectTimeoutError(<urllib3.contrib.socks.SOCKSHTTPSConnection object at 0x0000020B202CA5C0>, 'Connection to api.ipgeolocation.io timed out. (connect timeout=4)'))
4.017355680465698  --  香港CN2-B - 三网适用 (hk-cn2-b.osoblanco.xyz:4936)
3 => 2.7471163272857666  --  133.18.202.21 Japan Chiyoda Asia/Tokyo   ---   日本IIJ-B - 三网适用
4 => 1.5322656631469727  --  89.31.126.187 Japan Shinagawa Asia/Tokyo   ---   日本IIJ-C - 可看Netflix / 三网适用
5 => 1.1946628093719482  --  103.99.179.16 United States Los Angeles America/Los_Angeles   ---   美国CN2-B - 可看

{'台湾GCE - 扣双倍流量 / 非原生IP / 电信联通适用 (tw-gce.osoblanco.xyz:4936)',
 '新加坡CN2-A - 三网适用 (sg-cn2-a.osoblanco.xyz:4936)',
 '日本IIJ - 三网适用 (jp-iij.osoblanco.xyz:4936)',
 '日本Vultr - 无限流量 / 三网适用 (jp-vultr.osoblanco.xyz:4936)',
 '美国CN2-A - 可看Netflix / 三网适用 (us-cn2.osoblanco.xyz:4936)',
 '香港-游戏专用 - 扣5倍流量 / 限速2Mbps (hk-gaming.osoblanco.xyz:4936)',
 '香港CN2-A - 三网适用 (hk-cn2-a.osoblanco.xyz:4936)',
 '香港CN2-B - 三网适用 (hk-cn2-b.osoblanco.xyz:4936)'}